# Decorator Application: Single Dispatch Generic Functions

In [1]:
from html import escape

In [2]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a,2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}<\li>'.format(html_escape(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items('<li>{0}={1}</li>'.format(k, v)
         for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [3]:
print(html_str("""this is a
multi line string
with special characters: 10 < 100"""))

In [4]:
print(html_int(255))

In [5]:
print(html_escape(3+10j))

In [6]:
from decimal import Decimal

In [7]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [8]:
htmlize(100)

'100(<i>0x64</i>)'

In [9]:
htmlize('python rocks!')

'python rocks!'

In [10]:
htmlize([1,2,3])

'<ul>\n<li>1<\\li>\n<li>2<\\li>\n<li>3<\\li>\n</ul>'

In [11]:
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python
rocks! 0 &lt; 1
<\li>
<li>(10, 20, 30)<\li>
<li>100<\li>
</ul>


In [12]:
from decimal import Decimal
from html import escape

In [13]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg, set):
        return html_set(arg)
    else:
        return html_escape(arg)
    
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a,2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}<\li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items('<li>{0}={1}</li>'.format(html_escape(k), htmlize(v))
         for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)

In [14]:
htmlize(100)

'100(<i>0x64</i>)'

In [15]:
print(htmlize(["""Python
rocks! 0 < 1
""", (10, 20, 30), 100]))

<ul>
<li>Python<br/>
rocks! 0 &lt; 1<br/>
<\li>
<li><ul>
<li>10(<i>0xa</i>)<\li>
<li>20(<i>0x14</i>)<\li>
<li>30(<i>0x1e</i>)<\li>
</ul><\li>
<li>100(<i>0x64</i>)<\li>
</ul>


In [16]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict
    }
    
    fn = registry.get(type(arg), registry[object])
    
    return fn(arg)

In [17]:
htmlize(100)

'100(<i>0x64</i>)'

In [18]:
print(htmlize([1,2,3]))

<ul>
<li>1(<i>0x1</i>)<\li>
<li>2(<i>0x2</i>)<\li>
<li>3(<i>0x3</i>)<\li>
</ul>


# section 115!

In the last section we have the fn htmlize() which takes in an arg and based on the type it looks up which function 
should be applied to that arg. 

In [19]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: html_dict
    }
    
    fn = registry.get(type(arg), registry[object])
    
    return fn(arg)

In [20]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [21]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [22]:
htmlize('1 < 100')

'1 &lt; 100'

In [23]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    registry[int] = lambda a: '{0}(<i>{1}</i>)'.format(a, str(hex(a)))
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')
    
    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    return inner

In [24]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [25]:
htmlize('1 < 100')

'1 &lt; 100'

In [26]:
htmlize(100)

'100(<i>0x64</i>)'

In [41]:
def singledispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorated.register = register
    #not good for user access... ok for debugging
    # ###decorated.registry = registry
    decorated.dispatch = dispatch
    return decorated

In [42]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [29]:
htmlize('1 < 100')

'1 &lt; 100'

In [30]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

In [46]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [47]:
html_int(3)

'3(<i>0x3</i>)'

In [48]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = ('<li>{0}<\li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [49]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [50]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [51]:
htmlize(100)

'100(<i>0x64</i>)'

In [52]:
htmlize.registry

AttributeError: 'function' object has no attribute 'registry'

In [53]:
html_sequence((1, 2, 3))

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [54]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [55]:
from numbers import Integral

In [56]:
class Person:
    pass

In [57]:
class Student(Person):
    pass

In [58]:
p = Student()

In [59]:
type(p)

__main__.Student

In [60]:
isinstance(p, Student)

True

In [61]:
isinstance(p, Person)

True

In [62]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [63]:
@htmlize.register(Integral)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [64]:
isinstance(10, Integral)

True

In [65]:
isinstance(True, Integral)

True

In [66]:
htmlize(10)

'10'

In [67]:
@htmlize.register(int)
@htmlize.register(bool)
def html_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [68]:
htmlize(10)

'10(<i>0xa</i>)'

In [69]:
htmlize(True)

'True(<i>0x1</i>)'

In [70]:
from collections.abc import Sequence

In [71]:
isinstance([1,2,3], Sequence)

True

In [72]:
isinstance((1,2,3), Sequence)

True

In [73]:
type([1, 2, 3]) is Sequence

False

# Part 3!

In [74]:
from functools import singledispatch

In [75]:
from numbers import Integral
from collections.abc import Sequence

In [76]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [77]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [79]:
htmlize.dispatch(str)

<function __main__.htmlize(a)>

In [81]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

In [82]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>})

In [83]:
htmlize.dispatch(int)

<function __main__.htmlize_integral_number(a)>

In [84]:
type(10)

int

In [85]:
isinstance(10, int)

True

In [86]:
isinstance(10, Integral)

True

In [87]:
isinstance(True, Integral)

True

In [88]:
htmlize.dispatch(bool)

<function __main__.htmlize_integral_number(a)>

In [89]:
htmlize(10)

'10(<i>0xa</i>)'

In [90]:
htmlize(True)

'True(<i>0x1</i>)'

In [91]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ('<li>{0}<\li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [92]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [93]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [94]:
isinstance('Python', Sequence)

True

In [95]:
htmlize('python')

RecursionError: maximum recursion depth exceeded in comparison

In [97]:
for s in 'python':
    print(s)

p
y
t
h
o
n


In [98]:
@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [99]:
htmlize('python 1 < 100')

'python 1 &lt; 100'

In [101]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [102]:
htmlize((1,2,3))

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [104]:
@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return '({0})'.format(', '.join(items))

In [105]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(l)>,
              str: <function __main__.html_str(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [106]:
htmlize((1,2,3))

'(1, 2, 3)'

In [107]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

In [108]:
@htmlize.register(Integral)
def _(a):
    return '{0}(<i>{1}</i>)'.format(a, str(hex(a)))

@htmlize.register(Sequence)
def _(l):
    items = ('<li>{0}<\li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(str)
def _(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [109]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__._(a)>,
              collections.abc.Sequence: <function __main__._(l)>,
              str: <function __main__._(s)>,
              tuple: <function __main__.html_tuple(t)>})

In [110]:
_

<function __main__._(s)>

In [114]:
htmlize.dispatch(Integral)
id(htmlize.dispatch(Integral))

1872432223504

In [115]:
htmlize.dispatch(str)
id(htmlize.dispatch(str))

1872432224592

In [116]:
htmlize.dispatch(Sequence)
id(htmlize.dispatch(Sequence))

1872432226088